# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### A. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score,make_scorer
from sklearn.neighbors import KNeighborsClassifier
import pickle
import warnings
warnings.simplefilter('ignore')

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


In [3]:
# load data from database
###engine = create_engine('sqlite:///notebooks/processeddata.db')
engine = create_engine('sqlite:///disaster.db')

df = pd.read_sql("SELECT * FROM disaster_response", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### B. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    tokens = word_tokenize(text)
    #####print(tokens)
    stemmed = [PorterStemmer().stem(word) for word in tokens if word not in stopwords.words("english")]

    return stemmed

### C. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. 

In [6]:

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(verbose=False)))
])
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

### D. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)#, random_state = 1)

In [8]:
%%time
np.random.seed(45)
pipeline.fit(X_train, Y_train)

Wall time: 19min 30s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000277EB266430>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(verbose=False)))])

### E. Testing the model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
def get_score(y_true, y_pred, target_names):
    df = pd.DataFrame()
    for i,target in enumerate(target_names):
        accuracy = accuracy_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i],average='micro')
        precision = precision_score(y_true[:, i], y_pred[:, i],average='micro')
        recall = recall_score(y_true[:, i], y_pred[:, i],average='micro')
        df = df.append({'index':target,'Accuracy':accuracy,'F1 Score':f1,'Precision':precision,'Recall':recall},ignore_index = True)
    return df

In [22]:
target_names = Y.columns

In [23]:
%%time
y_train_pred = pipeline.predict(X_train)
y_train_true = np.array(Y_train)

Wall time: 8min 48s


In [24]:
result_train_df = get_score(y_train_true, y_train_pred, target_names)
display(result_train_df)

,index,Accuracy,F1 Score,Precision,Recall
0,related,0.997625,0.997625,0.997625,0.997625
1,request,0.998737,0.998737,0.998737,0.998737
2,offer,0.999949,0.999949,0.999949,0.999949
3,aid_related,0.998585,0.998585,0.998585,0.998585
4,medical_help,0.999495,0.999495,0.999495,0.999495
5,medical_products,0.999798,0.999798,0.999798,0.999798
6,search_and_rescue,0.999848,0.999848,0.999848,0.999848
7,security,0.999747,0.999747,0.999747,0.999747
8,military,0.999798,0.999798,0.999798,0.999798
9,water,0.999899,0.999899,0.999899,0.999899


In [25]:
y_true = np.array(Y_test)
y_pred = pipeline.predict(X_test)

In [26]:
%%time
result_pred_df = get_score(y_true, y_pred, target_names)
display(result_pred_df)

,index,Accuracy,F1 Score,Precision,Recall
0,related,0.820070,0.820070,0.820070,0.820070
1,request,0.905108,0.905108,0.905108,0.905108
2,offer,0.994998,0.994998,0.994998,0.994998
3,aid_related,0.789601,0.789601,0.789601,0.789601
4,medical_help,0.922995,0.922995,0.922995,0.922995
5,medical_products,0.954980,0.954980,0.954980,0.954980
6,search_and_rescue,0.973321,0.973321,0.973321,0.973321
7,security,0.980900,0.980900,0.980900,0.980900
8,military,0.968774,0.968774,0.968774,0.968774
9,water,0.954525,0.954525,0.954525,0.954525


Wall time: 784 ms


### F. Improve the model
we use grid search to find better parameters. 

In [35]:
### Get the existing params
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x00000277EB266430>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(verbose=False)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x00000277EB266430>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(verbose=False)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 '

In [57]:
def mid_f1(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i], average='micro')
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [89]:
def func_x(i):
    j = i ** 3
    k = func_y(j) # changed
    return k

In [101]:
def func_y(m):
    k = m ** 3
    return k

In [117]:
from joblib import Parallel, delayed
parameters = {'vect__min_df': [1, 5],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[5, 10]}
scorer = make_scorer(mid_f1)
#####Parallel(n_jobs=8, verbose=3)(delayed(func_x)(i) for i in range(50))
cv = GridSearchCV(pipeline, param_grid = parameters,scoring = scorer,verbose = 10, n_jobs=1)

In [118]:
cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000277EB266430>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(verbose=False)))]),
             n_jobs=1,
             param_grid={'clf__estimator__min_samples_split': [5, 10],
                         'clf__estimator__n_estimators': [10, 25],
                         'tfidf__use_idf': [True, False],
                         'vect__min_df': [1, 5]},
             scoring=make_scorer(mid_f1), verbose=10)

In [119]:
%%time
np.random.seed(450)
tuned_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5; 1/16] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 1/5; 1/16] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.961 total time= 3.9min
[CV 2/5; 1/16] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 2/5; 1/16] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.958 total time= 4.1min
[CV 3/5; 1/16] START clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1
[CV 3/5; 1/16] END clf__estimator__min_samples_split=5, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=1;, score=0.957 total time= 4.2min
[CV 4/5; 1/16] START clf__estimator__min_samples_split=5, clf__estimator__n_estimato

In [120]:
print(tuned_model.best_params_)
print(cv.best_score_)

{'clf__estimator__min_samples_split': 10, 'clf__estimator__n_estimators': 25, 'tfidf__use_idf': True, 'vect__min_df': 5}
0.9628076225691979


### G. Test the model
We show the accuracy, precision, and recall of the tuned model.  

In [121]:
%%time
y_true = np.array(Y_test)
y_tuned_pred = tuned_model.predict(X_test)
result_tuned_df = get_score(y_true, y_tuned_pred, target_names)
display(result_tuned_df)

,index,Accuracy,F1 Score,Precision,Recall
0,related,0.817493,0.817493,0.817493,0.817493
1,request,0.900106,0.900106,0.900106,0.900106
2,offer,0.994695,0.994695,0.994695,0.994695
3,aid_related,0.778839,0.778839,0.778839,0.778839
4,medical_help,0.922541,0.922541,0.922541,0.922541
5,medical_products,0.956799,0.956799,0.956799,0.956799
6,search_and_rescue,0.972715,0.972715,0.972715,0.972715
7,security,0.980597,0.980597,0.980597,0.980597
8,military,0.969077,0.969077,0.969077,0.969077
9,water,0.958314,0.958314,0.958314,0.958314


Wall time: 1min 8s


In [122]:
%%time
display(result_pred_df.describe())
display(result_tuned_df.describe())

,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.948639,0.948639,0.948639,0.948639
std,0.048971,0.048971,0.048971,0.048971
min,0.789601,0.789601,0.789601,0.789601
25%,0.943080,0.943080,0.943080,0.943080
50%,0.956344,0.956344,0.956344,0.956344
75%,0.981886,0.981886,0.981886,0.981886
max,0.995149,0.995149,0.995149,0.995149


,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.948648,0.948648,0.948648,0.948648
std,0.050475,0.050475,0.050475,0.050475
min,0.778839,0.778839,0.778839,0.778839
25%,0.945506,0.945506,0.945506,0.945506
50%,0.958314,0.958314,0.958314,0.958314
75%,0.982265,0.982265,0.982265,0.982265
max,0.995149,0.995149,0.995149,0.995149


Wall time: 32.9 ms


### H. Try improving your model further. Here are a few ideas:
* we try other machine learning algorithms
* we add other features besides the TF-IDF

In [125]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

parameters_kn = {'tfidf__use_idf':[True, False],
                 'clf__estimator__weights':['uniform','distance'],
                 'clf__estimator__leaf_size':[25,35]
                }
cv = GridSearchCV(pipeline, param_grid = parameters_kn,scoring = scorer,verbose = 10, n_jobs=1)

np.random.seed(25)
kn_model = cv.fit(X_train, Y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True
[CV 1/5; 1/8] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True;, score=0.956 total time= 4.4min
[CV 2/5; 1/8] START clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True
[CV 2/5; 1/8] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True;, score=0.954 total time= 4.7min
[CV 3/5; 1/8] START clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True
[CV 3/5; 1/8] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True;, score=0.956 total time= 4.8min
[CV 4/5; 1/8] START clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True
[CV 4/5; 1/8] END clf__estimator__leaf_size=25, clf__estimator__weights=uniform, tfidf__use_idf=True;, score=0.955 total time=

In [126]:
print(kn_model.best_params_)
print(cv.best_score_)

{'clf__estimator__leaf_size': 25, 'clf__estimator__weights': 'uniform', 'tfidf__use_idf': False}
0.9562382524722883


In [127]:
y_true = np.array(Y_test)
y_kn_pred = kn_model.predict(X_test)
result_kn_df = get_score(y_true, y_kn_pred, target_names)
display(result_kn_df)
display(result_kn_df.describe())

,index,Accuracy,F1 Score,Precision,Recall
0,related,0.780658,0.780658,0.780658,0.780658
1,request,0.884190,0.884190,0.884190,0.884190
2,offer,0.994846,0.994846,0.994846,0.994846
3,aid_related,0.695013,0.695013,0.695013,0.695013
4,medical_help,0.919054,0.919054,0.919054,0.919054
5,medical_products,0.953161,0.953161,0.953161,0.953161
6,search_and_rescue,0.973624,0.973624,0.973624,0.973624
7,security,0.981204,0.981204,0.981204,0.981204
8,military,0.968319,0.968319,0.968319,0.968319
9,water,0.950584,0.950584,0.950584,0.950584


,Accuracy,F1 Score,Precision,Recall
count,35.000000,35.000000,35.000000,35.000000
mean,0.937647,0.937647,0.937647,0.937647
std,0.067085,0.067085,0.067085,0.067085
min,0.695013,0.695013,0.695013,0.695013
25%,0.928149,0.928149,0.928149,0.928149
50%,0.953464,0.953464,0.953464,0.953464
75%,0.981658,0.981658,0.981658,0.981658
max,0.995149,0.995149,0.995149,0.995149


### 9. Export your model as a pickle file

In [128]:
pickle.dump(tuned_model, open('disaster_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.